# Fermionic-algebra-based measurement optimization

Fermionic-algebra-based measurement optimization techniques improve the efficiency of the quantum measurement of the molecular electronic Hamiltonian. This quantum measurement is one of the remaining challenges for the practicality of the variational quantum eigensolver \[see [Phys. Rev. Research **4**, 033154 (2022)](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.033154)\]. Unlike the [qubit-algebra-based methods](./QubitMethods.ipynb), the fermionic-algebra-based methods introduced here uses the properties of the molecular electronic Hamiltonian (presented in the second quantized form):
$$
\hat{H} = \sum_{pq} h_{pq} \hat{a}^{\dagger}_{p} \hat{a}_{q} + \sum_{pqrs} g_{pqrs} \hat{a}^{\dagger}_{p} \hat{a}_{q} \hat{a}^{\dagger}_{r} \hat{a}_{s}.
$$
The main idea behind the measurement technique is that Hartree&ndash;Fock (HF) solvable parts can be transformed by orbital rotations into functions of occupation number operators $\hat{n}_{p} = \hat{a}^{\dagger}_{p} \hat{a}_{p}$. These $\hat{n}_{p}$'s can be measured directly on a digital quantum computer because they transform to $\hat{z}$ under the Jordan&ndash;Wigner transformation.
More details of measurement techniques based on partitioning $\hat{H}$ into HF solvable parts can be found in [PRX Quantum **2**, 040320 (2021)](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.2.040320). 

The following two techniques are available within Tequila:

1. Low-rank (LR) decomposition [npj Quantum Inf. **7**, 23 (2021)](https://www.nature.com/articles/s41534-020-00341-7)

2. Fluid fermionic fragments (F$^{3}$) [Quantum **7**, 889 (2023)](https://quantum-journal.org/papers/q-2023-01-03-889/pdf/)

One potential drawback of LR is the high number of measurements required to obtain the expectation value of $\hat{H}$ as demonstrated in [PRX Quantum **2**, 040320 (2021)](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.2.040320). 
F$^3$ exploits the properties of the HF solvable parts obtained by LR to significantly reduce this required number of measurements. The resulting F$^3$-LR technique has lower quantum measurement costs than even the best qubit-algebra-based techniques \[see [Quantum **7**, 889 (2023)](https://quantum-journal.org/papers/q-2023-01-03-889/pdf/)\].  

## Common to both LR and F$^3$

### Setting up the test Hamiltonian
For concreteness, we demonstrate LR and F$^3$ using H$_4$ in the STO-3G basis.

Many of Tequila's functions including the one for measuring the expectation value of $\hat{H}$ takes Tequila's QubitHamiltonian as an input. However, the fermionic-algebra-based measurement optimization techniques require the fermionic Hamiltonian in the second quantized form. We use OpenFermion's reverse_jordan_wigner function to obtain the fermionic Hamiltonian back from Tequila's QubitHamiltonian.

In [1]:
# Needs pyscf or psi4 installed:
# pip install pyscf
# tequila version needs to be > 1.8.3 or from devel branch
import tequila as tq
from tequila.hamiltonian import QubitHamiltonian
from tequila.grouping.fermionic_functions import n_elec
from openfermion import reverse_jordan_wigner
import numpy as np

def prepare_test_hamiltonian():
    '''
    Return a test hamiltonian.
    '''
    trafo = "JordanWigner"
    mol = tq.chemistry.Molecule(
                            geometry="H 0.0 0.0 0.0 \n H 0.0 0.0 1. \n H 0.0 0.0 2. \n H 0.0 0.0 3.",
                            basis_set="sto3g",
                            transformation=trafo,
                            backend='pyscf'
                                 )
    H = mol.make_hamiltonian()
    Hq = H.to_openfermion()
    Hferm = reverse_jordan_wigner(Hq)
    return mol, H, Hferm, len(Hq.terms) - 1

mol, H, Hferm, n_paulis = prepare_test_hamiltonian()
print("Number of Pauli products to measure: {}".format(n_paulis))

converged SCF energy = -2.09854593699776
Number of Pauli products to measure: 184


## Low-rank decomposition

### Running LR

To obtain HF solvable fragments using the LR decomposition, one can simply execute the following.
As outputs, we obtain the HF solvable parts as orbital rotations (orb_rot) and the cartan subalgebra polynomials: There is a single one-body term (cartan_obt) and multiple two-body terms (cartan_tbts). The suggested measurement allocation (meas_alloc) is the optimal sample (shot) allocation for each HF solvable fragment minimizing the number of required number of measurements. Optimal meas_alloc is computed using the configuration interaction singles and doubles (CISD) wavefunction.

In [2]:
from tequila.grouping.fermionic_methods import do_svd

orb_rots, cartan_obt, cartan_tbts, meas_alloc = do_svd(Hferm, n_elec("h4"))

Starting SVD routine
Diagonalizing two-body tensor
Truncating SVD for coefficients with magnitude smaller or equal to 8.898272270834066e-18, using 10 fragments
Finished SVD routine
Allocating measurements


### Number of required measurements in LR

In the following, we estimate the number of measurements (in millions) required to obtain the Hamiltonian expectation value with a millihartree accuracy. For this estimation, we suppose that one is interested in measuring $\langle \mathrm{FCI} | \hat{H} | \mathrm{FCI} \rangle$ because all succesful VQE algorithms should converge close to the FCI solution. To evaluate the measurement cost, we use Eq. (4) from [Quantum **7**, 889 (2023)](https://quantum-journal.org/papers/q-2023-01-03-889/pdf/). 

In the output, "Full variances" show the variance of each HF solvable fragments, and "Expectations" show their expectation values. The metric in Eq. (4) is shown by "Variance metric value" and "Exp value" is the expectation value of $\hat{H}$, obtained by summing the fragments' expectation values:
$$
\langle \mathrm{FCI} | \hat{H} | \mathrm{FCI} \rangle = \sum_{\alpha} \langle \mathrm{FCI} | \hat{H}_{\alpha} | \mathrm{FCI} \rangle,
$$
where $\hat{H}_{\alpha}$ are the HF solvable fragments.

In [3]:
from tequila.grouping.fermionic_functions import obt_to_ferm, convert_tbts_to_frags
from tequila.grouping.fermionic_methods import get_wavefunction, compute_and_print_ev_var

def cartan_to_fermionic_operator(cobt, ctbts, orb_rot):
    '''
    Turn Cartan polynomials into OpenFermion's FermionOperator.
    '''
    obt = np.einsum("pa, qb, ab", orb_rot[0], orb_rot[0], cobt)
    tbts = np.einsum("ipa, iqb, irc, isd, iabcd -> ipqrs", 
                     orb_rot[1:], orb_rot[1:], orb_rot[1:], orb_rot[1:], ctbts)
    ferm_ops = [obt_to_ferm(obt,True)] + convert_tbts_to_frags(tbts, True)
    return ferm_ops

ferm_ops = cartan_to_fermionic_operator(cartan_obt, cartan_tbts, orb_rots)
#Compute the number of required measurements [Eq.(4) from Quantum 7, 889 (2023)] using the FCI wavefunction.
_, psis_fci = get_wavefunction(H.to_openfermion(), "fci", "h4", n_elec("h4"), save=False)
compute_and_print_ev_var(psis_fci[0], Hferm, ferm_ops, meas_alloc=meas_alloc)
    

Full variances:
[1.25991914e-01 1.36077664e-01 1.42450177e-02 4.37352361e-02
 2.11796719e-02 2.34092987e-04 5.93277530e-05 3.37726148e-05
 1.48370444e-09 4.65833765e-10 2.16294055e-17]
Expectations
[-8.42898449e+00  3.60748346e+00  6.17420954e-02  1.48399801e-01
  1.09695039e-01  1.62661519e-02  1.88238558e-03  2.39898990e-02
  2.19853203e-05  1.48979274e-05  9.95470945e-09]
Variance metric value is 1.5050679489201466
Exp value is -2.1663875200311833


## Fluid fermionic fragments (with LR fragments)

### Running F$^3$-LR and obtaining the number of required measurements

Unlike the LR decomposition, which did not require any input options, one can specify several options for F$^3$. Here is a summary of possible options:

1. mol_name (default null): Name of the molecule to perform FFF. Used for saving restart files.
2. fff_method (default R2): Full/R1/R2 \[see [Quantum **7**, 889 (2023)](https://quantum-journal.org/papers/q-2023-01-03-889/pdf/)\]. Full takes more classical computational time than R1 and R2, but has lower number of quantum measurements.
3. n_iter (default 5): Number of FFF iterations.
4. trunc_perc (default 100): For improved efficiency, CISD wavefunction can be truncated to include only a few significant slater determinants. trunc_perc specifies what percentage of the CISD wavefunction one wishes to retain. 
5. fff_thresh (default 1e-4): One can choose to only apply F$^3$ technique on a few of the HF solvable fragments. F$^3$ is only applied to the fragments with variances above fff_thresh. The larger the fff_thresh, the faster the algorithm, but the larger the quantum measurement cost. 

The options are passed to the do_fff function by using a dictionary (unspecified values are set to default):

In [4]:
options={"mol_name":"h4", "fff_method":"R2", "trunc_perc":99}

Unlike in LR, one can directly print the number of required measurements in F$^3$ by specifying metric_estim=True.

In [5]:
from tequila.grouping.fermionic_methods import do_fff
_ = do_fff(Hferm, n_elec("h4"), options=options, metric_estim=True)

Starting SVD routine
Diagonalizing two-body tensor
Truncating SVD for coefficients with magnitude smaller or equal to 8.898272270834066e-18, using 10 fragments
Finished SVD routine
Using 4 slater determinants: 0.9926785298742373% of the approximate WF
FCI info before optimization.
Full variances:
[1.25991914e-01 1.36077664e-01 1.42450177e-02 4.37352361e-02
 2.11796719e-02 2.34092987e-04 5.93277530e-05 3.37726148e-05
 1.48368937e-09 4.65833765e-10 2.16294055e-17]
Expectations
[-8.42898449e+00  3.60748346e+00  6.17420954e-02  1.48399801e-01
  1.09695039e-01  1.62661519e-02  1.88238558e-03  2.39898990e-02
  2.19852802e-05  1.48979274e-05  5.85181579e-08]
Variance metric value is 1.5048658096596963
Exp value is -2.166387471507828
Getting approximate variances
Applying F3 to 6 fragments out of 11
Computing necessary covariance estimates
Progress: iteration #1 out of 5
Progress: iteration #2 out of 5
Progress: iteration #3 out of 5
Progress: iteration #4 out of 5
Progress: iteration #5 out o

We can see that the number of measurements required to obtain the Hamiltonian expectation value has reduced by more than a factor of 2 from 1.5 million to 0.6 million by using the F$^3$ technique.

## Measuring the expectation value using a quantum circuit

Now let us use Tequila to measure the expectation value of a test wavefunction (wfn) on a quantum circuit. While the demonstration is only for F$^3$-LR, the LR decomposition can be used on its own by passing options_lr instead of options_f3_lr into tq.ExpectationValue. 

**Note that by default, reverse_jordan_wigner is used to obtain the fermionic Hamiltonian from Tequila's QubitHamiltonian to use a user-defined function instead, add {"revserse_H_transf":name_of_user_defined_func} to options_f3_lr.**

In [6]:
options_lr = {"method":"lr", "mol_name":"h4"}
options_f3_lr = {"method":"fff-lr"}
options_f3_lr.update(options)
print("Using {} as options".format(options_f3_lr))

#Prepare test wavefunction (wfn)
U = mol.make_ansatz(name="SPA", edges=[(0,1), (2,3)])
E = tq.ExpectationValue(H=H, U=U)
result = tq.minimize(E, silent=True)
wfn = tq.simulate(U, variables=result.variables)

e_f3 = tq.ExpectationValue(H=H, U=U, optimize_measurements=options_f3_lr)

#Print Benchmark Energy (unnecessary in real measurement).
result_f3 = tq.simulate(e_f3, result.variables)
print("Benchmark Energy:", result_f3)

compiled_f3 = tq.compile(e_f3)
# auto-100000 means automatically allocate 100000 samples (shots) 
# between measurable fragments.
exp_f3 = compiled_f3(result.variables, samples="auto-600000")

print("Energy measured with {}\u00b3-LR".format("F"), exp_f3)


Using {'method': 'fff-lr', 'mol_name': 'h4', 'fff_method': 'R2', 'trunc_perc': 99} as options
Using default reverse_H_transf, i.e., reverse_jordan_wigner.
Using saved Hamiltonian from SAVE/h4/. Run with a different mol_name if this is not desired.
Using saved LR decomposition saved in SAVE/h4/. Run with a different mol_name if this is not desired.
Using saved psi_cisd in SAVE/h4/. Run with a different mol_name if this is not desired.
Using 4 slater determinants: 0.9926785298742373% of the approximate WF
Getting approximate variances
Applying F3 to 6 fragments out of 11
Computing necessary covariance estimates
Progress: iteration #1 out of 5
Progress: iteration #2 out of 5
Progress: iteration #3 out of 5
Progress: iteration #4 out of 5
Progress: iteration #5 out of 5
Allocating measurements
Benchmark Energy: -3.636819839477539
Energy measured with F³-LR -3.6391637325286865
